## predict

In [12]:
# UltralyticsのYOLOv8を使用して画像推論を実行
from ultralytics import YOLO
import os
from pathlib import Path
import glob

print("ライブラリをインポートしました")

# パスの設定
model_path = r"D:\prog_py\Yolo_trainer\runs\train\train9\weights\best.pt"
source_path = "detect_images"

# モデルファイルと画像フォルダの存在確認
if os.path.exists(model_path):
    print(f"✓ モデルファイル確認: {model_path}")
else:
    print(f"✗ モデルファイルが見つかりません: {model_path}")

if os.path.exists(source_path):
    images = [f for f in os.listdir(source_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    print(f"✓ 画像フォルダ確認: {source_path} ({len(images)}枚の画像)")
else:
    print(f"✗ 画像フォルダが見つかりません: {source_path}")

# YOLOモデルの読み込み
model = YOLO(model_path)
print(f"✓ モデルを読み込みました")
print(f"モデルクラス数: {len(model.names)}")
print(f"クラス名: {list(model.names.values())}")


ライブラリをインポートしました
✗ モデルファイルが見つかりません: D:\prog_py\Yolo_trainer\runs\train\train9\weights\best.pt
✓ 画像フォルダ確認: detect_images (13枚の画像)


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x000001FBCB47CFE0>
Traceback (most recent call last):
  File "d:\prog_py\yolo12_detect\.venv\Lib\site-packages\torch\utils\data\dataloader.py", line 1618, in __del__
  File "d:\prog_py\yolo12_detect\.venv\Lib\site-packages\torch\utils\data\dataloader.py", line 1576, in _shutdown_workers
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\prog_py\\Yolo_trainer\\runs\\train\\train9\\weights\\best.pt'

In [ ]:
# 指定されたパラメータで画像推論を実行
print("推論を開始します...")

results = model.predict(
    source=source_path,      # 画像フォルダ: detect_images
    imgsz=1200,              # 画像サイズ: 800
    save_txt=True,          # テキストファイル保存: True
    line_width=1,           # バウンディングボックスの線の太さ: 1
    save=True,              # 推論結果の画像を保存
    conf=0.25,              # 信頼度のしきい値
    device='0',           # デバイス（GPUが利用可能な場合は'0'に変更）
    project='runs/detect',  # 結果保存ディレクトリ
    name='yolo_detection',  # 実験名 # 既存のディレクトリがある場合上書き
    retina_masks=True,
    agnostic_nms=True,
)

print("✓ 推論が完了しました！")
print(f"結果は runs/detect/yolo_detection に保存されました")


推論を開始します...

WARNING imgsz=[1200] must be multiple of max stride 32, updating to [1216]
image 1/13 d:\prog_py\yolo12_detect\detect_images\Image_000003.jpg: 1216x1120 74 pods, 93.1ms
image 2/13 d:\prog_py\yolo12_detect\detect_images\Image_000118.jpg: 1216x736 74 pods, 87.3ms
image 3/13 d:\prog_py\yolo12_detect\detect_images\Image_000188.jpg: 1216x704 52 pods, 84.4ms
image 4/13 d:\prog_py\yolo12_detect\detect_images\Image_000473.jpg: 1216x672 69 pods, 96.4ms
image 5/13 d:\prog_py\yolo12_detect\detect_images\Image_000565.jpg: 1216x512 70 pods, 80.5ms
image 6/13 d:\prog_py\yolo12_detect\detect_images\Image_000896.jpg: 1216x544 56 pods, 113.4ms
image 7/13 d:\prog_py\yolo12_detect\detect_images\Image_001075.jpg: 1216x896 39 pods, 111.7ms
image 8/13 d:\prog_py\yolo12_detect\detect_images\Image_001845.jpg: 1216x480 41 pods, 88.5ms
image 9/13 d:\prog_py\yolo12_detect\detect_images\Image_001851.jpg: 1216x608 15 pods, 81.0ms
image 10/13 d:\prog_py\yolo12_detect\detect_images\Image_001885.jpg: 121

In [ ]:
# dataset.yamlを使用したモデル検証
print("=== モデル検証を開始します ===\n")

# dataset.yamlのパス
dataset_yaml_path = r"D:\Data\pods\2025_yolo_pod\labels\YOLODataset\dataset.yaml"

# dataset.yamlファイルの存在確認
if os.path.exists(dataset_yaml_path):
    print(f"✓ データセット設定ファイル確認: {dataset_yaml_path}")
    
    # ファイルの内容を表示
    with open(dataset_yaml_path, 'r', encoding='utf-8') as f:
        content = f.read()
        print(f"\nデータセット設定:")
        print(content)
else:
    print(f"✗ データセット設定ファイルが見つかりません: {dataset_yaml_path}")
    print("検証を実行できません。")


=== モデル検証を開始します ===

✓ データセット設定ファイル確認: D:\Data\pods\2025_yolo_pod\labels\YOLODataset\dataset.yaml

データセット設定:
path: \\?\D:\Data\pods\2025_yolo_pod\labels\YOLODataset
train: images/train
val: images/val
test:

names:
    0: pod



In [ ]:
# 画像サイズ（imgsz）はmax stride 32の倍数でグリッドサーチ
if os.path.exists(dataset_yaml_path):
    print("🔍 画像サイズ（32の倍数, 1500まで）で検証を実行しています...")

    # 32の倍数で500から1504まで
    grid_imgsz = list(range(512, 1505, 32))

    for imgsz in grid_imgsz:
        print(f"\n--- 画像サイズ: {imgsz} で検証中 ---")
        validation_results = model.val(
            data=dataset_yaml_path,
            imgsz=imgsz,
            plots=True,
            batch=8,
            save_json=True,
            device='0',
            project='runs/val',
            name=f'val_imgs_{imgsz}',
            verbose=True,
        )
        metrics = getattr(validation_results, 'metrics', None)
        if metrics is not None:
            map_50_95 = getattr(metrics, 'mAP_0.5_0.95', None)
            precision = getattr(metrics, 'precision', None)
            recall = getattr(metrics, 'recall', None)
            print(f"  - imgsz={imgsz}: mAP@0.5-0.95={map_50_95}, Precision={precision}, Recall={recall}")
        else:
            print(f"  - imgsz={imgsz}: メトリクス取得不可")

else:
    print("❌ データセット設定ファイルが見つからないため、検証をスキップします。")


##　出力変換

In [13]:
from ultralytics import YOLO

# Load a model

model = YOLO(r"D:\prog_py\Yolo_trainer\runs\train\yolo11_split\weights\best.pt")  # load a custom trained model

# Export the model
model.export(format="onnx")

Ultralytics 8.3.203  Python-3.12.6 torch-2.6.0+cu124 CPU (Intel Core(TM) i7-10875H 2.30GHz)
 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
YOLO11s-seg summary (fused): 113 layers, 10,067,203 parameters, 0 gradients, 32.8 GFLOPs

PyTorch: starting from 'D:\prog_py\Yolo_trainer\runs\train\yolo11_split\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 37, 8400), (1, 32, 160, 160)) (19.7 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim>=0.1.67', 'onnxruntime-gpu'] not found, attempting AutoUpdate...

requirements: AutoUpdate success  37.7s
WARNING requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.19.1 opset 19...
ONNX: slimming with onnxslim 0.1.72...
ONNX: export success  42.3s, saved as 'D:\prog_py\Yolo_trainer\runs\train\yolo11_split\weights\best.onnx' (38.7 MB)

Export complete (43.1

'D:\\prog_py\\Yolo_trainer\\runs\\train\\yolo11_split\\weights\\best.onnx'